In [1]:
%cd '/Users/abhilashdhal/OmixHub/'

/Users/abhilashdhal/OmixHub


## 1. Imports

In [2]:
import gevent.monkey
gevent.monkey.patch_all(thread=False, select=False)
import pandas as pd
import numpy as np

import os
from importlib import reload
from flatten_json import flatten
from tqdm import tqdm 
import src.Engines.gdc_engine as gdc_engine
import src.Connectors.gcp_bigquery_utils as gcp_bigquery_utils
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
reload(gcp_bigquery_utils)
reload(gdc_engine)

<module 'src.Engines.gdc_engine' from '/Users/abhilashdhal/OmixHub/src/Engines/gdc_engine.py'>

In [3]:
from ipywidgets import widgets

## 1. Initialize gdc_engine to get data from GDC

In [24]:

params = {
    'files.experimental_strategy': 'RNA-Seq', 
    'data_type': 'Gene Expression Quantification'
}

gdc_eng_inst = gdc_engine.GDCEngine(**params)


In [25]:
metadata = gdc_eng_inst._get_rna_seq_metadata()

dict_keys(['files.experimental_strategy', 'data_type'])
file_id,file_name,submitter_id,experimental_strategy,data_type,platform,cases.case_id,cases.project.name,cases.project.project_id,cases.project.program.name,cases.project.program.program_id,cases.samples.sample_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.primary_diagnosis,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.treatments.treatment_or_therapy,cases.diagnoses.days_to_last_follow_up,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_death,cases.project.primary_site,analysis.workflow_type,cases.demographic.ethnicity,cases.demographic.gender,cases.demographic.race,cases.diagnoses.tissue_or_organ_of_origin,cases.exposures.bmi,cases.exposures.alcohol_history,cases.exposures.years_smoked,cases.samples.tissue_type,cases.samples.sample_typecases.samples.is_ffpe,cases.samples.preservation_method,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.age_at_diagnosis,cas

In [6]:
meta = metadata['metadata']

In [7]:
# Extract unique primary sites from metadata, remove None, and sort
list_of_primary_sites = sorted([site for site in meta['primary_site'].unique() if site is not None])



# # Create a dropdown widget
# primary_site_dropdown = widgets.Dropdown(
#     options=list_of_primary_sites,
#     description='Primary Site:',
#     disabled=False,
# )

# # Display the dropdown
# display(primary_site_dropdown)


In [8]:
# Count occurrences of each primary site
primary_site_counts = meta['primary_site'].value_counts()

# Create a list of tuples (site, count) for non-None sites, sorted by site name
list_of_primary_sites = sorted(
    [(site, count) for site, count in primary_site_counts.items() if site is not None],
    key=lambda x: x[0]
)

# # Create dropdown options with site names and counts
# dropdown_options = [f"{site} ({count})" for site, count in list_of_primary_sites]

# # Create a dropdown widget
# primary_site_dropdown = widgets.Dropdown(
#     options=dropdown_options,
#     description='Primary Site:',
#     disabled=False,
# )

# # Display the dropdown
# display(primary_site_dropdown)

## 2. Fetch data from GDC for single or multiple primary sites and upload to bigquery

In [ ]:

primary_sites = [
    'Bone'
]

## Initialize BigQueryUtils with your project
bq_utils = gcp_bigquery_utils.BigQueryUtils(project_id='rnaseq-451105', credentials_path='/Users/abhilashdhal/OmixHub/personal_docs/rnaseq-451105-663734b00f0a.json')
bq_utils.project_exists()
dataset_id = 'gdcdata16022025'
## Enter New Table Name or Existing table name
table_name = 'bonernaseqexpr'

## create table_id as dataset_id + table_name
table_id =  dataset_id + '.' + table_name

## Give Schema of your table to be created or updated 
schema = [
    bigquery.SchemaField("case_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("file_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("expr_unstr_count", "INTEGER", mode="REPEATED"),
    bigquery.SchemaField("tissue_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("sample_type", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("primary_site", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("tissue_or_organ_of_origin", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("age_at_diagnosis", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("primary_diagnosis", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("race", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("gender", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("group_identifier", "INTEGER", mode="NULLABLE")
]

## Create table with partitioning and clustering
bq_utils.create_bigquery_table_with_schema(
    table_id=table_id, schema=schema, partition_field="group_identifier", clustering_fields=["primary_site", "tissue_type"]
)

## Specify the Kind of Downstream Analysis you want to perform
downstream_analysis = 'ML'
for site in tqdm(primary_sites):
    json_object, gene_cols = gdc_eng_inst.make_count_data_for_bq(site, downstream_analysis='DE', format='json')

    # Load data into BigQuery
    job = bq_utils.load_json_data(json_object, schema, table_id)
    job.result()  # Wait for the job to complete
    print(f"Data for {site} loaded successfully.")

I0000 00:00:1739687324.116840 11924424 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Table rnaseq-451105.gdcdata16022025.BoneRNASeqExpr already exists


  0%|          | 0/1 [00:00<?, ?it/s]

dict_keys(['files.experimental_strategy', 'data_type'])
file_id,file_name,submitter_id,experimental_strategy,data_type,platform,cases.case_id,cases.project.name,cases.project.project_id,cases.project.program.name,cases.project.program.program_id,cases.samples.sample_id,cases.diagnoses.last_known_disease_status,cases.diagnoses.primary_diagnosis,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.treatments.treatment_or_therapy,cases.diagnoses.days_to_last_follow_up,cases.diagnoses.age_at_diagnosis,cases.diagnoses.days_to_death,cases.project.primary_site,analysis.workflow_type,cases.demographic.ethnicity,cases.demographic.gender,cases.demographic.race,cases.diagnoses.tissue_or_organ_of_origin,cases.exposures.bmi,cases.exposures.alcohol_history,cases.exposures.years_smoked,cases.samples.tissue_type,cases.samples.sample_typecases.samples.is_ffpe,cases.samples.preservation_method,cases.diagnoses.tumor_stage,cases.diagnoses.tumor_grade,cases.diagnoses.age_at_diagnosis,cas








  0%|          | 0/1 [00:58<?, ?it/s]


ValueError: DataFrame columns must be unique for orient='records'.

In [13]:
gene_cols.shape

(60660,)

In [21]:
## Initialize BigQueryUtils with your project
bq_utils = gcp_bigquery_utils.BigQueryUtils(project_id='rnaseqml')
gene_cols_df = pd.DataFrame(gene_cols)
gene_cols_df.columns = ['gene_id']
schema = [
    bigquery.SchemaField("gene_id", "STRING", mode="NULLABLE")]
## Initialize BigQueryUtils with your project
bq_utils = gcp_bigquery_utils.BigQueryUtils(project_id='rnaseqml')
table_id = 'rnaseqml.rnaseqexpression.gene_ids_ordered'
## Create table with partitioning and clustering
bq_utils.create_bigquery_table_with_schema(
    table_id=table_id, schema=schema
)
# Load data into BigQuery
job = bq_utils.upload_df_to_bq(table_id, gene_cols_df)
job.result()  # Wait for the job to complete
print(f"Data for {site} loaded successfully.")

I0000 00:00:1726381924.760514 1749342 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1726381925.269214 1749342 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1726381925.523471 1749342 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1726381925.526964 1749342 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1726381925.784694 1749342 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1726381926.036745 1749342 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Created table rnaseqml.rnaseqexpression.gene_ids_ordered
Data for Bone loaded successfully.


In [23]:
gene_ids_or_gene_cols = pd.read_csv('/Users/abhilashdhal/Projects/personal_docs/data/Transcriptomics/data/gene_annotation/gene_id_to_gene_name_mapping.csv')

In [24]:
gene_ids_or_gene_cols.head()

,gene_id,gene_name
0,ENSG00000258011.2,HMGA1P3
1,ENSG00000186792.17,HYAL3
2,ENSG00000234551.2,LINC01309
3,ENSG00000270818.1,AL445675.1
4,ENSG00000008323.15,PLEKHG6


In [27]:
gene_cols_df.merge(gene_ids_or_gene_cols).to_csv('./personal_docs/data/gene_id_name_mapping.csv', index=False)

In [31]:
gene_ids_or_gene_cols

,gene_id,gene_name
0,ENSG00000258011.2,HMGA1P3
1,ENSG00000186792.17,HYAL3
2,ENSG00000234551.2,LINC01309
3,ENSG00000270818.1,AL445675.1
4,ENSG00000008323.15,PLEKHG6
...,...,...
60655,ENSG00000267077.1,AC020663.2
60656,ENSG00000109111.15,SUPT6H
60657,ENSG00000253088.1,Y_RNA
60658,ENSG00000255401.1,AC021269.2
